In [71]:
# Notebook to process data from TriTime timeclock data.
import pandas as pd
import lib.tritime as tt


In [72]:
# First get a list of every user in the system
badges = tt.get_badges()
# Create some empty lists to hold the data that we'll put into a DataFrame
bcol, dcol, tin, tout, tdiff = [], [], [], [], []
# Now let's loop through everybody in the system.
for badge_num, badge in badges.items():
    dn = badge['display_name']
    # Grab their punnch in/out data one by one
    punch_data = tt.read_punches(badge_num)
    # Now for every punch in/out pair, let's add it to our lists
    for p in punch_data:
        if 'ts_in' not in p or 'ts_out' not in p:
            continue  # And flag as an error?
        bcol.append(badge_num)
        dcol.append(dn)
        tin.append(p['ts_in'])
        tout.append(p['ts_out'])
        tdiff.append(p['duration'])

# With all the data in lists, let's create a DataFrame
tdf = pd.DataFrame({'badge': bcol, 'display_name': dcol,
                    'time_in': tin, 'time_out': tout, 'duration': tdiff})

# Now we 'cast' the data to the correct types; fixing datetime here
tdf['time_in'] = pd.to_datetime(tdf['time_in'])
tdf['time_out'] = pd.to_datetime(tdf['time_out'])
# Now let's make a new column that tells of what week of the year it is
tdf['week_number'] = tdf['time_in'].dt.isocalendar().week
# And because nobody likes to think of dates by week number of year (except
# perhaps farmers), let's make a new column that tells us the start of the week
# as a date.
tdf['sow'] = (
    tdf['time_in'] - pd.to_timedelta(tdf['time_in'].dt.dayofweek, unit='d')
).dt.date
tdf['year'] = tdf['time_in'].dt.isocalendar().year
tdf.to_csv('test.csv', index=False)
tdf.to_parquet('test.parquet', index=False)
tdf

,badge,display_name,time_in,time_out,duration,week_number,sow,year
0,895,Allendale,2024-09-03 12:07:33,2024-09-03 12:12:16,283.0,36,2024-09-02,2024
1,895,Allendale,2024-09-04 19:26:25,2024-09-04 19:26:39,14.0,36,2024-09-02,2024
2,895,Allendale,2024-09-04 19:46:08,2024-09-05 10:32:53,53205.0,36,2024-09-02,2024
3,9013,JC,2024-09-09 13:48:00,2024-09-11 09:53:04,158704.0,37,2024-09-09,2024
4,chad,Chad Potinsky,2024-08-27 19:17:32,2024-08-27 19:17:38,6.0,35,2024-08-26,2024
...,...,...,...,...,...,...,...,...
60,justin,Justin Buist,2024-09-05 11:51:56,2024-09-05 12:03:23,687.0,36,2024-09-02,2024
61,justin,Justin Buist,2024-09-05 17:28:47,2024-09-07 17:34:34,173147.0,36,2024-09-02,2024
62,justin,Justin Buist,2024-09-07 17:53:35,2024-09-11 09:53:06,316771.0,36,2024-09-02,2024
63,justin,Justin Buist,2024-09-11 09:53:09,2024-09-11 09:53:19,10.0,37,2024-09-09,2024


In [73]:
# Now let's get a list of all the users in the system, just one row for each
# user.
udf = tdf[['badge', 'display_name']].drop_duplicates()
udf

,badge,display_name
0,895,Allendale
3,9013,JC
4,chad,Chad Potinsky
14,david,David Austin
15,elijah,Elijah H
22,justin,Justin Buist


In [74]:
# Now use the raw data to create a DataFrame that has the total time worked
# for each user for every week of the year.
wdf = (
    tdf.drop(['display_name', 'time_in', 'time_out'], axis=1)
       .groupby(['badge', 'year', 'week_number', 'sow'])
       .sum()
).reset_index()
wdf

,badge,year,week_number,sow,duration
0,895,2024,36,2024-09-02,53502.0
1,9013,2024,37,2024-09-09,158704.0
2,chad,2024,35,2024-08-26,309.0
3,chad,2024,36,2024-09-02,558335.0
4,david,2024,35,2024-08-26,513453.0
5,elijah,2024,36,2024-09-02,489956.0
6,justin,2024,35,2024-08-26,514704.0
7,justin,2024,36,2024-09-02,636985.0
8,justin,2024,37,2024-09-09,515.0


In [75]:
# Now let's merge the two DataFrames together to get a final DataFrame that
# makes for a decent report
merged = pd.merge(wdf, udf, on=['badge'], how='left')
merged['duration_hours'] = merged['duration'] / 3600
merged

,badge,year,week_number,sow,duration,display_name,duration_hours
0,895,2024,36,2024-09-02,53502.0,Allendale,14.861667
1,9013,2024,37,2024-09-09,158704.0,JC,44.084444
2,chad,2024,35,2024-08-26,309.0,Chad Potinsky,0.085833
3,chad,2024,36,2024-09-02,558335.0,Chad Potinsky,155.093056
4,david,2024,35,2024-08-26,513453.0,David Austin,142.625833
5,elijah,2024,36,2024-09-02,489956.0,Elijah H,136.098889
6,justin,2024,35,2024-08-26,514704.0,Justin Buist,142.973333
7,justin,2024,36,2024-09-02,636985.0,Justin Buist,176.940278
8,justin,2024,37,2024-09-09,515.0,Justin Buist,0.143056


In [76]:
# Last we 'pivot' the data so that the weeks are new each a column
finaldf = merged.pivot(index='display_name',
                       columns='sow',
                       values='duration_hours').fillna(0).reset_index(level=0)

finaldf

sow,display_name,2024-08-26,2024-09-02,2024-09-09
0,Allendale,0.000000,14.861667,0.000000
1,Chad Potinsky,0.085833,155.093056,0.000000
2,David Austin,142.625833,0.000000,0.000000
3,Elijah H,0.000000,136.098889,0.000000
4,JC,0.000000,0.000000,44.084444
5,Justin Buist,142.973333,176.940278,0.143056


In [77]:
finaldf.to_excel('report.xlsx', index=False)
